In [53]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os                       # operating system tools (check files)

from census import Census # This is new...

import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Point, Polygon # also needed

import pyarrow as pa
import pyarrow.parquet as pq

##########################
# Then this stuff below allows us to make a nice inset


from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

## Interactive Covid-19 Cases by County Map With County Time Series

The code below creates an interactive map illustrating Covid-19 cases by county and new the time series of cases by county. The end result generates a live map so that when you use the hover tool and the hover over a county, it plots the time series of case in addition to  reporting the  county name, cases, deaths, and population. There is also a zoom in feature so one can zoom into, say NYC region.  

The underlying Covid-19 data is from reporting done by the New York Times and hosted in their data repository on github.

### Overview of code

Thansk to the [@nytimes](https://github.com/nytimes/covid-19-data). This proved to be very easy given my existing code associated with my project on the [trade war](https://github.com/mwaugh0328/consumption_and_tradewar). The code proceeds in several steps:

1. Grabs the nytimes data. Their repository provides detailed explanations regarding geography etc. NYC in particular is treated as one region, not by county. 

2. Merge nytimes data with the geopandas dataframe, add in US Census data (currently just population), then some simple cleaning to prepare for maping.

3. Create a live/html map with hover tool and zoom using Bokeh's packages. The output will be an .html file with can be posted on a website etc. It is currently posted at [https://mwaugh0328.github.io/covid-19-map/us_covid_map.html](https://mwaugh0328.github.io/covid-19-map/us_covid_map.html)

### Prerequisites

- The code below assumes you have the correct shapefiles. If not, or you don't know what I'm talking about run ``download_shapefiles.ipynb`` which pull the correct shapefiles to make the map.

- The hover tool here incoperates figures generated from the ``county-by-time.ipynb`` notebook.

---

### Step \#1. Grab data from [nytimes repository](https://github.com/nytimes/covid-19-data)

Thank you for organizing this. It's very simple...

In [54]:
df = pd.read_csv("https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv")

In [55]:
# For NYC we will assing it the Mannhatten fips code
# Then below to the other new york city counties we 
# assing them the same case and death data as New York city as a whole

nyc = df.county == "New York City"
df.loc[nyc,"fips"] = 36061

In [56]:
# Convert the date to a datetime object
df["date"] = pd.to_datetime(df["date"], format = "%Y-%m-%d")

In [57]:
latest_date = df.date.max()

In [58]:
latest_date = latest_date.strftime("%m/%d/%Y")

In [59]:
df.set_index("date", inplace = True)

In [60]:
dfall = df.loc[latest_date].copy()

**Important data note** The nytimes dataset reports **cumulative** cases, not the number reported each day. The very first version of this code messed this up. Now it is correct.

In [61]:
dfall[dfall.county == "New York City"]

,county,state,fips,cases,deaths
date,,,,,
2020-05-17,New York City,New York,36061.0,197486,20214


In [62]:
total = dfall.cases.sum()

print("Total US Covid-19 Cases", total, "as of", latest_date)

Total US Covid-19 Cases 1493599 as of 05/17/2020


In [63]:
total_deaths = dfall.deaths.sum()

print("Total US Covid-19 Deaths", total_deaths, "as of", latest_date)

Total US Covid-19 Deaths 89504 as of 05/17/2020


In [64]:
dfall[dfall["fips"] == 36061]

,county,state,fips,cases,deaths
date,,,,,
2020-05-17,New York City,New York,36061.0,197486,20214


In [65]:
dfall.dropna(subset = ["fips"], inplace = True)

In [66]:
file_path = os.getcwd()

doc_path = file_path + "\\docs\\"

html_path = "https://mwaugh0328.github.io/covid-19-map/cases/"

dfall["cases_file_location"] = html_path + "county_" + dfall.fips.astype(int).astype(str) + ".png"

html_path = "https://mwaugh0328.github.io/covid-19-map/deaths/"

dfall["deaths_file_location"] = html_path + "county_" + dfall.fips.astype(int).astype(str) + ".png"
#dfall["file_location"] = doc_path + "county_" + dfall.fips.astype(int).astype(str) + ".png"

In [67]:
dfall.head()

,county,state,fips,cases,deaths,cases_file_location,deaths_file_location
date,,,,,,,
2020-05-17,Autauga,Alabama,1001.0,110,4,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...
2020-05-17,Baldwin,Alabama,1003.0,254,8,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...
2020-05-17,Barbour,Alabama,1005.0,81,1,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...
2020-05-17,Bibb,Alabama,1007.0,50,1,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...
2020-05-17,Blount,Alabama,1009.0,46,1,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...


In [68]:
map_projection = "epsg:2163"

In [69]:
cwd = os.getcwd()

county_shape = cwd + "\\shapefiles\\lake\\ne_10m_lakes.shx"

lake_map = gpd.read_file(county_shape)

lake_map = lake_map.to_crs({'init': map_projection})

C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [70]:
cwd = os.getcwd()

county_shape = cwd + "\\shapefiles\\land\\ne_50m_land.shx"

land_map = gpd.read_file(county_shape)

land_map = land_map.to_crs({'init': map_projection})

land_map = land_map.iloc[0:1200]

C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [71]:
cwd = os.getcwd()

county_shape = cwd + "\\shapefiles\\county\\tl_2017_us_county.shx"

us_map = gpd.read_file(county_shape)

us_map = us_map.to_crs({'init': map_projection})

C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [72]:
us_map["geometry"] = us_map["geometry"].simplify(2000)
# This was important. The geometry in the tigerline file si
# too fine, orginal map was 350mb. simply basicly simplifies the geometry,
# making the map take up less memory and load faster. Still not sure
# what the number exactly means.

In [73]:
us_map["area_fips"] = (us_map.STATEFP.astype(str) + us_map.COUNTYFP.astype(str)).astype(int)

---

###  Step \#2 Merge and Clean

The next couple of cells download the requisite shapefiles from the US census. They are unzipped in a folder called shapefiles and then county. So they are assuming some structure behind your folder setup. 

In [74]:
us_map.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,area_fips
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477641638,10701538,+41.9158651,-096.7885168,"POLYGON ((246290.041 -328619.667, 245963.100 -...",31039
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-1770686.984 398793.664, -1772831.10...",53069
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016761648,29147345,+34.3592729,-104.3686961,"POLYGON ((-422754.447 -1210188.449, -442958.76...",35011
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169252486,22867561,+40.7835474,-096.6886584,"POLYGON ((259511.654 -444450.123, 259130.939 -...",31109
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645186,1718484,+40.1764918,-098.0468422,"POLYGON ((146978.459 -544334.301, 146407.623 -...",31129


In [75]:
us_map = us_map.merge(dfall, left_on='area_fips',
                      right_on = "fips", how = "left", indicator = True)

The cell below fills in the NYC region with the NYC values. So in the hover map, when you hover over say Brookly, it will show the value for the NYC region. I then added a data note column that will inform the reader of this.

In [76]:
#Fill in Queens (36081), Bronx (36005, Richmond(36085), Brooklyn (36047)
nyc_counties = [36081,36005,36085,36047]

us_map.loc[us_map.area_fips.isin(nyc_counties), "deaths"] = us_map.loc[us_map.area_fips == 36061,"deaths"].values[0]

us_map.loc[us_map.area_fips.isin(nyc_counties), "cases"] = us_map.loc[us_map.area_fips == 36061,"cases"].values[0]

us_map.loc[us_map.area_fips.isin(nyc_counties), "cases_file_location"] = us_map.loc[us_map.area_fips == 36061,
                                                                                    "cases_file_location"].values[0]

us_map.loc[us_map.area_fips.isin(nyc_counties), "deaths_file_location"] = us_map.loc[us_map.area_fips == 36061,
                                                                                     "deaths_file_location"].values[0]


us_map["Notes"] = ""

all_nyc_counties = [36081,36005,36085,36047,36061]

us_map.loc[us_map.area_fips.isin(all_nyc_counties), "Notes"] = "NYC counties are treated as one region"

In [77]:
all_nyc_counties = [36081,36005,36085,36047,36061]

us_map.loc[us_map.area_fips.isin(all_nyc_counties)]

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,area_fips,county,state,fips,cases,deaths,cases_file_location,deaths_file_location,_merge,Notes
1399,36,085,00974141,36085,Richmond,Richmond County,06,H6,G4020,408,...,36085,NaN,NaN,NaN,197486.0,20214.0,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...,left_only,NYC counties are treated as one region
2333,36,081,00974139,36081,Queens,Queens County,06,H6,G4020,408,...,36081,NaN,NaN,NaN,197486.0,20214.0,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...,left_only,NYC counties are treated as one region
2409,36,047,00974122,36047,Kings,Kings County,06,H6,G4020,408,...,36047,NaN,NaN,NaN,197486.0,20214.0,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...,left_only,NYC counties are treated as one region
2446,36,061,00974129,36061,New York,New York County,06,H6,G4020,408,...,36061,New York City,New York,36061.0,197486.0,20214.0,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...,both,NYC counties are treated as one region
3162,36,005,00974101,36005,Bronx,Bronx County,06,H6,G4020,408,...,36005,NaN,NaN,NaN,197486.0,20214.0,https://mwaugh0328.github.io/covid-19-map/case...,https://mwaugh0328.github.io/covid-19-map/deat...,left_only,NYC counties are treated as one region


In [78]:
#us_map[["NAME","NAMELSAD","county","state", "cases"]].head(48)

I'm from Alaska, so this next step is always painfull. Drop states and territories who are not part of the "lower 48", contiguous US. Still not clear how fix this in geopandas.

In [79]:
us_map.set_index("STATEFP", inplace = True)

drop_list = ["02","15","72","78","69","66","60",]

us_map.drop(drop_list, inplace = True)

In [80]:
state_shape = cwd + "\\shapefiles\\state\\tl_2017_us_state.shx"

state_map = gpd.read_file(state_shape)

state_map = state_map.to_crs({'init': map_projection})

state_map["geometry"] = state_map["geometry"].simplify(200)

C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [81]:
state_fp_dict = dict(zip(state_map.STATEFP, state_map.STUSPS))

In [82]:
state_map.set_index("STATEFP", inplace = True)

drop_list = ["02","15","72","78","69","66","60",]

state_map.drop(drop_list, inplace = True)

In [83]:
us_map.reset_index(inplace = True)

In [84]:
us_map["STSPS"] = us_map["STATEFP"].map(state_fp_dict)

In [85]:
us_map["NAME"] = us_map["NAME"] + " County, " + us_map["STSPS"]

In [86]:
us_map.set_index("STATEFP", inplace = True)

In [87]:
#us_map["cases"].replace("nan", np.nan, inplace = True)

#us_map["cases"] = us_map["cases"].str.replace(',','')

#us_map["cases"] = us_map["cases"].astype(int)

us_map["cases_label"] = us_map["cases"].round(0)

us_map["cases_label"] = us_map["cases_label"].map('{:,.0f}'.format)

us_map["deaths_label"] = us_map["deaths"].round(0)

us_map["deaths_label"] = us_map["deaths_label"].map('{:,.0f}'.format)

#### Add Census Data

Here I add this in. MY api key is posted here, please be respectfull of it.

In [88]:
my_api_key = '34e40301bda77077e24c859c6c6c0b721ad73fc7'
# This is my api_key, # don't abuse this.

c = Census(my_api_key)
# This will create an object c which has methods associated with it.
# We will see  these below.

code = ("NAME","B01001_001E") 
# Get more stuff from the cencuss if we want...

county_2017 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'county:*'}, year=2017))
                                         # Same deal, but we specify county then the wild card
                                         # On the example page, there are ways do do this, only by state
        
county_2017 = county_2017.rename(columns = {"B01001_001E":"2017_population"})

county_2017["GEOFIPS"] = (county_2017["state"] + county_2017["county"]).astype(int)

county_2017["2017_population"] = county_2017["2017_population"].astype(float)

county_2017.set_index(["GEOFIPS"], inplace = True)

In [89]:
county_2017.head()

,NAME,2017_population,state,county
GEOFIPS,,,,
72047,"Corozal Municipio, Puerto Rico",34933.0,72,047
72095,"Maunabo Municipio, Puerto Rico",11297.0,72,095
72111,"Peñuelas Municipio, Puerto Rico",21661.0,72,111
72113,"Ponce Municipio, Puerto Rico",148863.0,72,113
72131,"San Sebastián Municipio, Puerto Rico",38970.0,72,131


In [90]:
us_map = us_map.merge(county_2017[["2017_population"]], 
                      left_on='area_fips', right_on = "GEOFIPS", how = "left")

In [91]:
us_map["pop_label"] = us_map["2017_population"].map('{:,.0f}'.format)

In [92]:
q_cases = [0,1,5,10,100,250,500,1000,5000,10000,np.inf]

In [93]:

us_map["q_cases"]= pd.cut(us_map["cases"],q_cases, labels=range(0,10))
us_map["q_deaths"]= pd.cut(us_map["deaths"],q_cases, labels=range(0,10))

us_map["q_cases"].fillna(0, inplace = True)
us_map["q_deaths"].fillna(0, inplace = True)


In [94]:
us_map["cases_label"].replace("nan", "N.R.", inplace = True)
us_map["deaths_label"].replace("nan", "N.R.", inplace = True)

The final issue is to create the borders associated with the great lakes. The issue is that the tigerline shape files just extend boundries into the waterline. And the associated coastline maps are (i) not polygons and (ii) looked messed up. So per above, I found another shape file of lakes. Then I will use geopandas built in set operations to essentially "punch out" the holes associated with the great lakes. 

[https://geopandas.org/set_operations.html](https://geopandas.org/set_operations.html)

**Note** for teaching purposes, it's worth messing around with the ``how`` operations to understand what is going on here.

In [95]:
# This operation will take care of the coastline

us_map = gpd.overlay(us_map, land_map,  how='intersection')

In [96]:
great_lakes = ["Lake Superior", "Lake Michigan", "Lake Erie","Lake Superior""Lake Huron"]

us_map = gpd.overlay(us_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
# this is the key operation...note that the order matters on the difference operation. output is same 

In [97]:
state_map = gpd.overlay(state_map, land_map,  how='intersection')

state_map = gpd.overlay(state_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
# Do the same thing for the state map

In [98]:
#panda_path = "https://mwaugh0328.github.io/covid-19-map/angry_panda.gif"

#us_map["cases_file_location"].fillna(panda_path, inplace = True)

#crona_path = "https://media.giphy.com/media/dVuyBgq2z5gVBkFtDc/giphy.gif"

#us_map["deaths_file_location"].fillna(crona_path, inplace = True)

In [99]:
fill_list = ["https://mwaugh0328.github.io/covid-19-map/angry_panda.gif", 
             "https://media.giphy.com/media/dVuyBgq2z5gVBkFtDc/giphy.gif",
            "https://media.giphy.com/media/JRsY1oIVA7IetTkKVO/giphy.gif"]

us_map["deaths_file_location"].fillna(pd.Series(np.random.choice(fill_list, size=len(us_map.index))), inplace = True)

us_map["cases_file_location"].fillna(pd.Series(np.random.choice(fill_list, size=len(us_map.index))), inplace = True)

---

### Step \# 3. Create Live/html map

This took a huge amount of effort to all sort out form bokeh's documentation and other resources. It works and generates something really great in my opinion. 

Note, need to have a folder called ``docs`` below the working directory. It will save the ``.html`` file there. Then to see the map, just open the ``.html`` file in a web browswer.

**Updated** the code was modified to create ``tabs`` where the user could [select the measure](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html). In this case it will creat two tabs one for cases, one for deaths. This is very simple. Just great two mapping objects and then combine them using this ``tab`` function.

**Update II** now the hover tool incoperates a plot of the time series. Essentially, I closely studied and followed the example [here](https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#custom-tooltip) within the bokeh documentation.

In [100]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models import Title
from bokeh.layouts import gridplot

from bokeh.plotting import figure, save
from bokeh.models import Panel, Tabs

from bokeh.resources import CDN
from bokeh.embed import file_html

from bokeh.io import output_file, show
from bokeh.models import Div

# Input GeoJSON source that contains features for plotting
#geosource = GeoJSONDataSource(geojson = us_map.to_json())

This is cell containst stuff that is common across the two plots

In [101]:
state_geosource = GeoJSONDataSource(geojson = state_map.to_json())

#coast_geosource = GeoJSONDataSource(geojson = coast_map.to_json())

geosource = GeoJSONDataSource(geojson = us_map.to_json())

palette = brewer['RdBu'][10]

#https://docs.bokeh.org/en/latest/docs/reference/palettes.html

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 9)

tick_labels = {2:str(q_cases[2]), 
              4:str(q_cases[4]), 6:str(q_cases[6]), 8 :str(q_cases[9])}

This cell contains the first "tab" which is number of cases

In [102]:
title = "Covid-19 Cases by County as of " + latest_date + ", Total Cases: " + f"{total:,d}"



color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = tick_labels,
                     major_tick_line_alpha = .25)

# Create figure object.
p = figure( 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "box_zoom, reset")

descip = "Colorbar by # of covid-19 cases; Hover tool plots cases by day."
descip = descip + "Data from https://github.com/nytimes/covid-19-data"
p.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
p.add_layout(Title(text=title, text_font_size="11pt"), 'above')

author = "Created by Michael Waugh, NYU-Stern and NBER, www.waugheconomics.com"
p.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')


p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.

states = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'q_cases',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

state_line = p.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.5)

#coast_line = p.multi_line('xs','ys', source = coast_geosource,
#                   line_color = "black", 
#                   line_width = 0.5)
# Create hover tool

TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95;">
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@cases_file_location" height="200" alt="@cases_file_location" width="350"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Population: @pop_label &nbsp &nbsp Cases: @cases_label &nbsp &nbsp 
            Deaths: @deaths_label</span>
        </div>
    </div>
"""

p.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

#### Some features to make it a bit nicer. 

p.axis.visible = False
p.background_fill_color = "grey"
p.background_fill_alpha = 0.25

p.border_fill_color = "#F5F5F5"
color_bar.background_fill_color = "#F5F5F5"

p.toolbar.autohide = True

p.add_layout(color_bar, "right")

## Send to doc file, create a webpage from doc file on github
# then had weebly webiste point to that .html file. That's how
# I got this to work.
###################################3
#source = ColumnDataSource(source = geosource)

#https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#custom-tooltip

#show(pbar)
#######################################33


In [103]:
title_death = "Covid-19 Deaths by County as of " + latest_date + ", Total Deaths: " + f"{total_deaths:,d}"

color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = tick_labels,
                     major_tick_line_alpha = .25)

# Create figure object.
pdth = figure( 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "box_zoom, reset")

descip = "Colorbar by # of covid-19 cases; Hover tool plots cases by day."
descip = descip + " Data from https://github.com/nytimes/covid-19-data"
pdth.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
pdth.add_layout(Title(text=title_death, text_font_size="11pt"), 'above')

author = "Created by Michael Waugh, NYU-Stern and NBER, www.waugheconomics.com"
pdth.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')

pdth.xgrid.grid_line_color = None
pdth.ygrid.grid_line_color = None
# Add patch renderer to figure.

states = pdth.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'q_deaths',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

state_line = pdth.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.5)

TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95;">
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@deaths_file_location" height="200" alt="@deaths_file_location" width="350"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Population: @pop_label &nbsp &nbsp Cases: @cases_label &nbsp &nbsp 
            Deaths: @deaths_label</span>
        </div>
    </div>
"""

pdth.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

#### Some features to make it a bit nicer. 

pdth.axis.visible = False
pdth.background_fill_color = "grey"
pdth.background_fill_alpha = 0.25

pdth.border_fill_color = "#F5F5F5"
color_bar.background_fill_color = "#F5F5F5"

pdth.toolbar.autohide = True

pdth.add_layout(color_bar, "right")

## Send to doc file, create a webpage from doc file on github
# then had weebly webiste point to that .html file. That's how
# I got this to work.

In [104]:
tab1 = Panel(child=p, title="Cases")
tab2 = Panel(child=pdth, title="Deaths")

tabs = Tabs(tabs=[ tab1, tab2 ])

file_path = os.getcwd()

doc_path = file_path +"\\docs"

outfp = doc_path + "\\us_covid_map.html"

# Save the map
save(tabs, outfp)

# Not sure if this is important, but seemed to start working once
# I ran it
html = file_html(tabs, CDN, outfp)

C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\io\saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\mwaugh\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\io\saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
